
---
title: "Análisis de Riesgo de Diabetes"
---

# Introducción

La diabetes es una condición crónica que afecta a millones de personas en todo el mundo, con consecuencias importantes para la salud y la economía. Los factores de riesgo incluyen, entre otros, el sobrepeso, la inactividad física y la predisposición genética. La detección temprana del riesgo de diabetes es esencial para prevenir la aparición de la enfermedad y minimizar sus efectos.

Este análisis tiene como objetivo encontrar reglas de asociación en perfiles de individuos con diabetes. Los datos provienen del Behavioral Risk Factor Surveillance System (BRFSS) de 2015, una encuesta de salud pública que recopila información sobre factores de riesgo relacionados con la salud en adultos en los Estados Unidos.

Para este análisis, utilizaremos Python y el paquete de pandas para manipular y analizar los datos.

## Carga de Librerías

Primero, instalaremos y cargaremos las librerías necesarias para este análisis.



In [1]:
# Lectura del dataset proyectos.csv
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder
import seaborn as sns
import matplotlib.pyplot as plt


## Carga de Datos

A continuación, cargaremos los datos del archivo `diabetes_binary_5050split_health_indicators_BRFSS2015.csv`, el cual debe estar en el mismo directorio que este notebook. Verificaremos las primeras filas para entender la estructura del conjunto de datos.



In [ ]:
#Carga de datos
data = pd.read_csv('../data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')
data = data.drop(columns=['CholCheck', 'AnyHealthcare'])
# Ver nombres de columnas y tipos de datos
print(data.dtypes)

# Mostrar las primeras filas del conjunto de datos
print(data.head())

# Mostrar las dimensiones del conjunto de datos (filas, columnas)
print(data.shape)


Diabetes_binary         float64
HighBP                  float64
HighChol                float64
BMI                     float64
Smoker                  float64
Stroke                  float64
HeartDiseaseorAttack    float64
PhysActivity            float64
Fruits                  float64
Veggies                 float64
HvyAlcoholConsump       float64
NoDocbcCost             float64
GenHlth                 float64
MentHlth                float64
PhysHlth                float64
DiffWalk                float64
Sex                     float64
Age                     float64
Education               float64
Income                  float64
dtype: object
   Diabetes_binary  HighBP  HighChol   BMI  Smoker  Stroke  \
0              0.0     1.0       0.0  26.0     0.0     0.0   
1              0.0     1.0       1.0  26.0     1.0     1.0   
2              0.0     0.0       0.0  26.0     0.0     0.0   
3              0.0     1.0       1.0  28.0     1.0     0.0   
4              0.0     0.0       0.0


# Preprocesamiento de Datos

En esta sección, realizaremos el preprocesamiento básico de los datos, incluyendo la conversión de columnas binarias (0-1) a variables booleanas, lo cual facilita el análisis.



In [3]:
# Identificar las columnas binarias
binary_columns = ['Diabetes_binary', 'HighBP', 'HighChol', 'Smoker', 
                  'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 
                  'Veggies', 'HvyAlcoholConsump', 'NoDocbcCost', 
                  'DiffWalk', 'Sex']

# Convertir variables binarias de float a booleano
data[binary_columns] = data[binary_columns].astype(bool)


# Crear una representación de transacciones
transactions = []

for _, row in data.iterrows():
    transaction = []
    
    # Agregar las variables binarias como nombres de columna cuando el valor es True
    for col in binary_columns:
        if row[col]:
            transaction.append(col)
    
    # Convertir variables categóricas en ítems únicos
    transaction.append(f"BMI_{row['BMI']}")
    transaction.append(f"GenHlth_{row['GenHlth']}")
    transaction.append(f"MentHlth_{row['MentHlth']}")
    transaction.append(f"PhysHlth_{row['PhysHlth']}")
    transaction.append(f"Age_{row['Age']}")
    transaction.append(f"Education_{row['Education']}")
    transaction.append(f"Income_{row['Income']}")
    
    transactions.append(transaction)

# Transformar las transacciones a formato binario para fpgrowth
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [12]:
print (df.columns)

Index(['Age_1.0', 'Age_10.0', 'Age_11.0', 'Age_12.0', 'Age_13.0', 'Age_2.0',
       'Age_3.0', 'Age_4.0', 'Age_5.0', 'Age_6.0',
       ...
       'PhysHlth_4.0', 'PhysHlth_5.0', 'PhysHlth_6.0', 'PhysHlth_7.0',
       'PhysHlth_8.0', 'PhysHlth_9.0', 'Sex', 'Smoker', 'Stroke', 'Veggies'],
      dtype='object', length=187)


### FP-GROWTH


El algoritmo FP-Growth es una técnica eficiente para descubrir patrones frecuentes en conjuntos de datos sin generar combinaciones explícitas. Su principal ventaja es el uso de un árbol de patrones frecuentes (FP-tree) para representar transacciones de forma compacta, reduciendo la memoria y el tiempo de procesamiento, especialmente en grandes volúmenes de datos. FP-Growth es escalable, evita la generación de candidatos, y es particularmente eficaz en datos dispersos donde pocas combinaciones ocurren con frecuencia. Estas características lo convierten en una opción superior para manejar datos complejos y voluminosos.

En el análisis de diabetes, síntomas y estilo de vida, FP-Growth puede descubrir relaciones frecuentes entre hábitos y diagnósticos, identificar patrones de coocurrencia de síntomas y proporcionar bases para recomendaciones personalizadas. Su eficiencia y capacidad para extraer patrones complejos hacen de FP-Growth una herramienta ideal para extraer conocimientos significativos de datos clínicos grandes y multifacéticos.

In [ ]:
# Aplicar el algoritmo FP-Growth
frequent_itemsets_fp = fpgrowth(df, min_support=0.01, use_colnames=True, max_len=3)

# Mostrar los conjuntos de ítems frecuentes
print(frequent_itemsets_fp)

In [5]:
# Generar reglas de asociación

rules = association_rules(frequent_itemsets_fp,len(frequent_itemsets_fp), metric="confidence", min_threshold=0.7)

# Ordenar las reglas por confianza
rules_sorted = rules.sort_values(by="confidence", ascending=False)

# Mostrar las mejores reglas por confianza
print("\n==== Mejores reglas con mayor confianza ====")
print(rules_sorted.head(10))

# Filtrar las mejores reglas según la confianza para la categoría Diabetes
def get_top_rules_by_confidence(rules, consequent, top_n=5):
    filtered_rules = rules[rules['consequents'].apply(lambda x: consequent in str(x))]
    sorted_rules = filtered_rules.sort_values(by="confidence", ascending=False)
    return sorted_rules.head(top_n)

if rules_sorted is not None:
    # 4. Buscar reglas relacionadas con Diabetes_binary
    diabetes_rules = rules_sorted[rules_sorted['consequents'].apply(
        lambda x: any('Diabetes_binary' in str(item) for item in x)
    )]
    


==== Mejores reglas con mayor confianza ====
                       antecedents     consequents  antecedent support  \
126           (Fruits, Income_8.0)       (Veggies)            0.191578   
2396       (GenHlth_1.0, BMI_23.0)  (PhysActivity)            0.011034   
511   (Education_6.0, GenHlth_1.0)  (PhysActivity)            0.064689   
504          (Fruits, GenHlth_1.0)       (Veggies)            0.082074   
1985        (Income_8.0, BMI_22.0)       (Veggies)            0.014089   
1107   (Fruits, HvyAlcoholConsump)       (Veggies)            0.022860   
516      (GenHlth_1.0, Income_8.0)  (PhysActivity)            0.057857   
53         (Fruits, Education_6.0)       (Veggies)            0.247864   
1854            (Fruits, BMI_20.0)       (Veggies)            0.012279   
1306             (Fruits, Age_3.0)       (Veggies)            0.017612   

      consequent support   support  confidence      lift  representativity  \
126             0.788774  0.175989    0.918630  1.164630     

In [6]:
print("\nEstadísticas generales:")
print(f"Total de reglas: {len(rules)}")
print(f"Reglas con Diabetes_binary: {len(diabetes_rules)}")
print(f"Reglas con Diabetes_binary:")
print(diabetes_rules.sort_values('confidence', ascending=False)
        [['antecedents', 'consequents', 'confidence', 'support']]
        .head(10)
        .to_string())


Estadísticas generales:
Total de reglas: 2575
Reglas con Diabetes_binary: 201
Reglas con Diabetes_binary:
                                antecedents        consequents  confidence   support
2069    (HeartDiseaseorAttack, GenHlth_5.0)  (Diabetes_binary)    0.854381  0.028136
2042                (GenHlth_5.0, Age_10.0)  (Diabetes_binary)    0.845630  0.010539
2056                (HighChol, GenHlth_5.0)  (Diabetes_binary)    0.842351  0.048450
2005                  (GenHlth_5.0, HighBP)  (Diabetes_binary)    0.834172  0.053938
1491       (HeartDiseaseorAttack, DiffWalk)  (Diabetes_binary)    0.829824  0.060771
463                   (Stroke, GenHlth_5.0)  (Diabetes_binary)    0.819804  0.013000
203   (HeartDiseaseorAttack, PhysHlth_30.0)  (Diabetes_binary)    0.819473  0.027740
473          (Stroke, HeartDiseaseorAttack)  (Diabetes_binary)    0.816775  0.023143
2196  (HeartDiseaseorAttack, MentHlth_30.0)  (Diabetes_binary)    0.815945  0.011727
2231              (Income_2.0, GenHlth_5.0)

### APRIORI

El algoritmo Apriori es una opción valiosa para el análisis de datos sobre diabetes, síntomas y estilo de vida porque permite descubrir reglas de asociación que revelan relaciones significativas entre diferentes variables. Estas reglas, del tipo "si condición A, entonces condición B", ofrecen patrones interpretables que facilitan la identificación de factores de riesgo comunes o combinaciones de síntomas asociados con el diagnóstico de diabetes.

Además, Apriori permite ajustar umbrales de soporte y confianza, lo que ayuda a filtrar patrones irrelevantes y destacar solo las relaciones más significativas en el contexto clínico. Su enfoque es especialmente útil para manejar datos categóricos y discretos, como registros médicos, donde las relaciones entre síntomas, antecedentes familiares y hábitos de vida son esenciales para el diagnóstico personalizado. Aunque puede ser más intensivo en recursos que otros algoritmos, su capacidad para generar reglas claras y comprensibles lo convierte en una herramienta adecuada para mejorar la toma de decisiones en el manejo de la diabetes.

In [ ]:
# Ejecutar Apriori
frequent_itemsets_ap = apriori(df, 0.01, use_colnames=True, max_len=3)

# Mostrar los conjuntos de ítems frecuentes
print(frequent_itemsets_ap)

In [8]:
# Generar reglas de asociación

rules = association_rules(frequent_itemsets_ap,len(frequent_itemsets_ap), metric="confidence", min_threshold=0.7)

# Ordenar las reglas por confianza
rules_sorted = rules.sort_values(by="confidence", ascending=False)

# Mostrar las mejores reglas por confianza
print("\n==== Mejores reglas con mayor confianza ====")
print(rules_sorted.head(10))

# Filtrar las mejores reglas según la confianza para la categoría Diabetes
def get_top_rules_by_confidence(rules, consequent, top_n=5):
    filtered_rules = rules[rules['consequents'].apply(lambda x: consequent in str(x))]
    sorted_rules = filtered_rules.sort_values(by="confidence", ascending=False)
    return sorted_rules.head(top_n)

if rules_sorted is not None:
    # 4. Buscar reglas relacionadas con Diabetes_binary
    diabetes_rules = rules_sorted[rules_sorted['consequents'].apply(
        lambda x: any('Diabetes_binary' in str(item) for item in x)
    )]
    


==== Mejores reglas con mayor confianza ====
                       antecedents     consequents  antecedent support  \
1897          (Fruits, Income_8.0)       (Veggies)            0.191578   
788        (GenHlth_1.0, BMI_23.0)  (PhysActivity)            0.011034   
1780  (Education_6.0, GenHlth_1.0)  (PhysActivity)            0.064689   
1859         (Fruits, GenHlth_1.0)       (Veggies)            0.082074   
763         (Income_8.0, BMI_22.0)       (Veggies)            0.014089   
1882   (Fruits, HvyAlcoholConsump)       (Veggies)            0.022860   
1955     (GenHlth_1.0, Income_8.0)  (PhysActivity)            0.057857   
1777       (Fruits, Education_6.0)       (Veggies)            0.247864   
719             (Fruits, BMI_20.0)       (Veggies)            0.012279   
482              (Fruits, Age_3.0)       (Veggies)            0.017612   

      consequent support   support  confidence      lift  representativity  \
1897            0.788774  0.175989    0.918630  1.164630     

In [9]:
print("\nEstadísticas generales:")
print(f"Total de reglas: {len(rules)}")
print(f"Reglas con Diabetes_binary: {len(diabetes_rules)}")
print(f"Reglas con Diabetes_binary:")
print(diabetes_rules.sort_values('confidence', ascending=False)
        [['antecedents', 'consequents', 'confidence', 'support']]
        .head(10)
        .to_string())


Estadísticas generales:
Total de reglas: 2575
Reglas con Diabetes_binary: 201
Reglas con Diabetes_binary:
                                antecedents        consequents  confidence   support
1417    (HeartDiseaseorAttack, GenHlth_5.0)  (Diabetes_binary)    0.854381  0.028136
186                 (GenHlth_5.0, Age_10.0)  (Diabetes_binary)    0.845630  0.010539
1420                (HighChol, GenHlth_5.0)  (Diabetes_binary)    0.842351  0.048450
1419                  (GenHlth_5.0, HighBP)  (Diabetes_binary)    0.834172  0.053938
1322       (HeartDiseaseorAttack, DiffWalk)  (Diabetes_binary)    0.829824  0.060771
1431                  (Stroke, GenHlth_5.0)  (Diabetes_binary)    0.819804  0.013000
1448  (HeartDiseaseorAttack, PhysHlth_30.0)  (Diabetes_binary)    0.819473  0.027740
1451         (Stroke, HeartDiseaseorAttack)  (Diabetes_binary)    0.816775  0.023143
1445  (HeartDiseaseorAttack, MentHlth_30.0)  (Diabetes_binary)    0.815945  0.011727
1422              (Income_2.0, GenHlth_5.0)

## Conclusión 

#### Análisis de las 10 reglas de asociación con mayor confianza

Todas estas reglas gozan de una confianza superiores a 0.80. De estas podemos obtener un perfil común en pacientes con diabetes/prediabetes:  

    - Los pacientes consideran tener una mala salud general.
    - Los pacientes han padecido enfermedad coronaria (CHD) o infarto de miocardio (IM).  
    - Los pacientes tienen entre 65-70 años.   
    - Los pacientes tienen colesterol alto.  
    - Los pacientes son hipertensos.
    - Los pacientes tienen dificultad para caminar y subir escaleras.
    - Los pacientes han sufrido un derrame cerebral.
    - Los pacientes han padecido de enfermedades o lesiones físicas los 30 días del pasado mes. 
    - Los pacientes han tenido una mala salud mental los 30 días del pasado mes. 
    - Los pacientes tienen bajos ingresos.

Así mismo, se pueden observar relaciones entre el consumo de frutas y verduras con un buen estado de salud y un BMI entre 20.0-23.0